In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import userdata


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = 'dragneel11'
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')


In [ ]:
!kaggle datasets download -d hijest/genre-classification-dataset-imdb

 60% 25.0M/41.7M [00:00<00:00, 106MB/s] 
100% 41.7M/41.7M [00:00<00:00, 130MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/genre-classification-dataset-imdb.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('/content/Genre Classification Dataset/train_data.txt',delimiter=':::',header=None, names=['name','genre','summary'], engine='python')
test_data = pd.read_csv('/content/Genre Classification Dataset/test_data.txt',delimiter=':::',header=None,names=['name', 'summary'],engine='python')

In [ ]:
train_data.head()

,name,genre,summary
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [ ]:
combined_data = pd.concat([test_data,train_data])

In [ ]:
combined_data = combined_data.drop('genre',axis=1)

In [ ]:
combined_data.head()

,name,summary
1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
5,Er nu zhai (1955),Before he was known internationally as a mart...


In [ ]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108414 entries, 1 to 54214
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   name     108414 non-null  object
 1   summary  108414 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def preprocess_text(text):
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Define a function to lemmatize a list of tokens
    def lemmatize_tokens(tokens):
        return [lemmatizer.lemmatize(token) for token in tokens]

    # Lowercase the text
    data = text.lower()

    # Remove punctuation
    data = re.sub(r'[^\w\s]+', '', data)

    # Tokenize the text
    tokens = nltk.word_tokenize(data)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize tokens
    lemmatized_tokens = lemmatize_tokens(filtered_tokens)

    # Join the tokens back into a string
    data = ' '.join(lemmatized_tokens)

    return data


In [ ]:
combined_data['summary'] = combined_data['summary'].apply(preprocess_text)

In [ ]:
combined_data.head()

,name,summary
1,Edgar's Lunch (1998),lr brane love life car apartment job especiall...
2,La guerra de papá (1977),spain march 1964 quico naughty child three bel...
3,Off the Beaten Track (2010),one year life albin family shepherd north tran...
4,Meu Amigo Hindu (2015),father died hasnt spoken brother 10 year serio...
5,Er nu zhai (1955),known internationally martial art superstar br...


In [ ]:
combined_data['tokenized_summary'] = combined_data['summary'].apply(word_tokenize)

In [ ]:
combined_data.head()

,name,summary,tokenized_summary
1,Edgar's Lunch (1998),lr brane love life car apartment job especiall...,"[lr, brane, love, life, car, apartment, job, e..."
2,La guerra de papá (1977),spain march 1964 quico naughty child three bel...,"[spain, march, 1964, quico, naughty, child, th..."
3,Off the Beaten Track (2010),one year life albin family shepherd north tran...,"[one, year, life, albin, family, shepherd, nor..."
4,Meu Amigo Hindu (2015),father died hasnt spoken brother 10 year serio...,"[father, died, hasnt, spoken, brother, 10, yea..."
5,Er nu zhai (1955),known internationally martial art superstar br...,"[known, internationally, martial, art, superst..."


In [ ]:
import gensim
model = gensim.models.Word2Vec(sentences=combined_data['tokenized_summary'].tolist(), vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
print(model.wv.most_similar('life'))

[('existence', 0.6648565530776978), ('relationship', 0.5770552754402161), ('routine', 0.5707352757453918), ('happiness', 0.5383937358856201), ('everything', 0.5277513861656189), ('destiny', 0.5168910026550293), ('living', 0.5167617797851562), ('seemingly', 0.5137726068496704), ('ordinary', 0.5112424492835999), ('lifestyle', 0.5104236006736755)]


In [ ]:
model.train(combined_data['tokenized_summary'].tolist(), total_examples=len(combined_data['tokenized_summary'].tolist()), epochs=model.epochs)

(30653624, 31138000)

In [ ]:
model.save("word2vec.model")

In [ ]:
def get_summary_vector(model, tokenized_summary):
    # Filter the list of tokens to make sure all of them are in the model's vocabulary
    tokens = [word for word in tokenized_summary if word in model.wv.key_to_index]

    # If the summary has no tokens in the model's vocabulary, return a zero vector
    if len(tokens) == 0:
        return np.zeros(model.vector_size)

    # Retrieve the vectors for each token in the summary
    word_vectors = [model.wv[word] for word in tokens]

    # Calculate the mean vector for the summary and return it
    summary_vector = np.mean(word_vectors, axis=0)
    return summary_vector

In [ ]:
train_data['summary'] = train_data['summary'].apply(preprocess_text)
test_data['summary'] = test_data['summary'].apply(preprocess_text)

In [ ]:
train_data.head()

,name,genre,summary
1,Oscar et la dame rose (2009),drama,listening conversation doctor parent 10yearold...
2,Cupid (1997),thriller,brother sister past incestuous relationship cu...
3,"Young, Wild and Wonderful (1980)",adult,bus empty student field trip museum natural hi...
4,The Secret Sin (1915),drama,help unemployed father make end meet edith twi...
5,The Unrecovered (2007),drama,film title refers unrecovered body ground zero...


In [ ]:
train_data['tokenized_summary'] = train_data['summary'].apply(word_tokenize)
test_data['tokenized_summary'] = test_data['summary'].apply(word_tokenize)

In [ ]:
test_data.head()

,name,summary,tokenized_summary
1,Edgar's Lunch (1998),lr brane love life car apartment job especiall...,"[lr, brane, love, life, car, apartment, job, e..."
2,La guerra de papá (1977),spain march 1964 quico naughty child three bel...,"[spain, march, 1964, quico, naughty, child, th..."
3,Off the Beaten Track (2010),one year life albin family shepherd north tran...,"[one, year, life, albin, family, shepherd, nor..."
4,Meu Amigo Hindu (2015),father died hasnt spoken brother 10 year serio...,"[father, died, hasnt, spoken, brother, 10, yea..."
5,Er nu zhai (1955),known internationally martial art superstar br...,"[known, internationally, martial, art, superst..."


In [ ]:
train_data['summary_vector'] = train_data['tokenized_summary'].apply(lambda x: get_summary_vector(model, x))
test_data['summary_vector'] = test_data['tokenized_summary'].apply(lambda x: get_summary_vector(model, x))


In [ ]:
test_data.head()

,name,summary,tokenized_summary,summary_vector
1,Edgar's Lunch (1998),lr brane love life car apartment job especiall...,"[lr, brane, love, life, car, apartment, job, e...","[0.5054405, -0.35254708, -0.1761063, -0.152459..."
2,La guerra de papá (1977),spain march 1964 quico naughty child three bel...,"[spain, march, 1964, quico, naughty, child, th...","[0.4090151, -0.06652626, -0.5603606, -0.912500..."
3,Off the Beaten Track (2010),one year life albin family shepherd north tran...,"[one, year, life, albin, family, shepherd, nor...","[0.5230343, 0.3956614, -0.024694858, -0.302766..."
4,Meu Amigo Hindu (2015),father died hasnt spoken brother 10 year serio...,"[father, died, hasnt, spoken, brother, 10, yea...","[0.073434144, -0.28648934, -0.17102568, -0.646..."
5,Er nu zhai (1955),known internationally martial art superstar br...,"[known, internationally, martial, art, superst...","[-0.27950808, 0.1724065, -0.029215286, -0.3285..."


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X = np.stack(train_data['summary_vector'].values)  # Features are the embeddings
y = train_data['genre'].values  # Target column

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.svm import SVC

# Initialize the Support Vector Classifier
svm_model = SVC()

# Train the model on the training data
svm_model.fit(X_train, y_train)

# Predict on the test data
y_pred = svm_model.predict(X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_pred))

Accuracy: 0.590150327400166


In [ ]:
test_data.head()

,name,summary,tokenized_summary,summary_vector
1,Edgar's Lunch (1998),lr brane love life car apartment job especiall...,"[lr, brane, love, life, car, apartment, job, e...","[0.5054405, -0.35254708, -0.1761063, -0.152459..."
2,La guerra de papá (1977),spain march 1964 quico naughty child three bel...,"[spain, march, 1964, quico, naughty, child, th...","[0.4090151, -0.06652626, -0.5603606, -0.912500..."
3,Off the Beaten Track (2010),one year life albin family shepherd north tran...,"[one, year, life, albin, family, shepherd, nor...","[0.5230343, 0.3956614, -0.024694858, -0.302766..."
4,Meu Amigo Hindu (2015),father died hasnt spoken brother 10 year serio...,"[father, died, hasnt, spoken, brother, 10, yea...","[0.073434144, -0.28648934, -0.17102568, -0.646..."
5,Er nu zhai (1955),known internationally martial art superstar br...,"[known, internationally, martial, art, superst...","[-0.27950808, 0.1724065, -0.029215286, -0.3285..."


In [ ]:
X_test = np.stack(test_data['summary_vector'].values)
test_data['genre_predict'] = svm_model.predict(X_test)

In [ ]:
solution_data = pd.read_csv('/content/Genre Classification Dataset/test_data_solution.txt',delimiter=':::',header=None,names=['name', 'genre','summary'],engine='python')

In [ ]:
test_data['actual_genre'] = solution_data['genre']

In [ ]:
test_data.sample(20)

,name,summary,tokenized_summary,summary_vector,genre_predict,actual_genre
35570,Journal d'un coopérant (2010),jeanmarc phaneuf unmarried electrical engineer...,"[jeanmarc, phaneuf, unmarried, electrical, eng...","[0.5309567, -0.013086731, -0.08172709, -0.0672...",drama,drama
5427,Going Clear: Scientology & the Prison of Beli...,devastating two hour documentary based lawrenc...,"[devastating, two, hour, documentary, based, l...","[0.004946709, 0.64219034, 0.09152871, 0.008101...",documentary,documentary
17100,Changement de trottoir (2004),summertime holiday time jeannot mechanic small...,"[summertime, holiday, time, jeannot, mechanic,...","[0.26853254, -0.53550225, 0.09137523, -0.48379...",comedy,short
47824,Zach's Lie (2018),jack osborne life change forever night men mas...,"[jack, osborne, life, change, forever, night, ...","[0.4613609, -0.41041464, -0.4156255, -0.256931...",drama,drama
18095,A tér (2013),look window control room district xii budapest...,"[look, window, control, room, district, xii, b...","[0.46940932, 0.16417658, -0.2927193, -0.390498...",documentary,documentary
31586,Article 58/4 (1996),documentary filmmaker pinchas schatz set deter...,"[documentary, filmmaker, pinchas, schatz, set,...","[0.1480717, 0.15339845, -0.32678425, -0.195177...",documentary,documentary
3827,The Most Dangerous Year (2018),2016 group washington state family transgender...,"[2016, group, washington, state, family, trans...","[0.2486012, 0.77049446, -0.0002836585, -0.6500...",documentary,documentary
7688,Finnlandia (2001),tragic loss husband young woman reconstructs c...,"[tragic, loss, husband, young, woman, reconstr...","[1.2094762, -0.33733255, -0.91672665, 0.128807...",drama,drama
46310,African Journey (1990),mainstar luke novak local hockey player living...,"[mainstar, luke, novak, local, hockey, player,...","[-0.016118277, -0.23011489, 0.13583563, -0.543...",drama,family
8056,Christian (2007),lauranne 50 life patrick 50 perverted director...,"[lauranne, 50, life, patrick, 50, perverted, d...","[0.17893443, -0.24212296, -0.19574611, -0.0640...",drama,drama


In [ ]:
accuracy = accuracy_score(test_data['actual_genre'], test_data['genre_predict'])
print(accuracy)

0.5871955719557196
